### S004 - Fuse results from a basic vector search AND a BM25 keyword search ###
- Supported strategies
    - S004_00 -> Fusion Basic
- Reranking
    - The Fusion retriever comes with a set of re-rankers
        - RECIPROCAL_RANK 
        - RELATIVE_SCORE 
        - DIST_BASED_SCORE 
        - SIMPLE 
    - There does not seem to be the ability to use external re-rankers (like Cohere)


In [1]:
# Fetch settings from config.py
import os
from config import set_environment 
set_environment()

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Only for notebook
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding


from datetime import datetime
import pandas as pd

Choose the model for generation

In [4]:
generation_llm_family = os.environ["GENERATION_LLM_FAMILY"]
generation_llm_model = os.environ["GENERATION_LLM_MODEL"]

if generation_llm_family == "OPENAI":
    Settings.llm = OpenAI(temperature=0, model=generation_llm_model)
elif generation_llm_family == "COHERE":
    Settings.llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=generation_llm_model,temperature=0)


Choose the model for embedding

In [5]:
embedding_llm_family = os.environ["EMBEDDING_LLM_FAMILY"]
embedding_llm_model = os.environ["EMBEDDING_LLM_MODEL"]

if embedding_llm_family == "OPENAI":
    Settings.embed_model = OpenAIEmbedding(model=embedding_llm_model,dimensions=512,)
elif embedding_llm_family == "COHERE":
    Settings.embed_model = CohereEmbedding(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model_name=embedding_llm_model,
    input_type="search_query",
)

Set the parameters for the run here

In [6]:
eval_name = os.environ["EVAL_NAME"]
eval_directory = os.environ["EVAL_DIRECTORY"]
eval_file = os.environ["EVAL_FILE"]
eval_questions = os.environ["EVAL_QUESTIONS"]
eval_results_dir = os.environ["EVAL_RESULTS_DIR"]
eval_quick_test = os.environ["EVAL_QUICK_TEST"]

rag_strategy = os.environ["RAG_STRATEGY"]

# Node Parser
chunk_size = int(os.environ["CHUNK_SIZE"])
chunk_overlap = 0.1 * chunk_size

similarity_top_k = int(os.environ["SIMILARITY_TOP_K"])
retriever_weights_string = os.environ["RETRIEVER_WEIGHTS"]
retriever_weights = [float(number) for number in retriever_weights_string.split('_')]
fusion_reranker = os.environ["FUSION_RERANKER"]

# Context Post Processor Settings
similarity_cutoff = float(os.environ["SIMILARITY_CUTOFF"])



Pick Strategy

In [7]:
if rag_strategy == "S004_00":
    rag_strategy_desc = "Fusion_Basic"
    run_id = f"{eval_name}_{rag_strategy}_GM_{generation_llm_model}_EM_{embedding_llm_model}_C_{chunk_size}_K_{similarity_top_k}_W_{retriever_weights_string}_RR_{fusion_reranker}_{datetime.today().strftime('%Y-%m-%d')}"

output_file = f"{eval_results_dir}/{run_id}.xlsx"  


Set up Token Counting

In [9]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode
)

Settings.callback_manager = CallbackManager([token_counter])
tokencount_df = pd.DataFrame()

Read the documents, create chunks, calculate embeddings, store in a vector database

In [10]:
reader = SimpleDirectoryReader(eval_directory)
documents = reader.load_data()

node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap = chunk_overlap)
nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(nodes):
    node.id_ = f"node-{idx}"

index = VectorStoreIndex(nodes, embed_model=Settings.embed_model, show_progress=True)

Generating embeddings:   0%|          | 0/82 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.a

In [11]:
tokencount_df['document_tokens'] = [token_counter.total_embedding_token_count]
token_counter.reset_counts()

Setup Retrieval and Response Generation

In [12]:
vector_retriever = index.as_retriever(similarity_top_k=similarity_top_k)#,llm=Settings.llm)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=similarity_top_k
)
retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=similarity_top_k,
    llm=Settings.llm,
    num_queries=1,  # set this to 1 to disable query generation
    mode=fusion_reranker,
    use_async=True,
    verbose=True,
    retriever_weights = retriever_weights,

    # query_gen_prompt="...",  # we could override the query generation prompt here
)


query_engine = RetrieverQueryEngine.from_args(retriever) 



Quick test of query engine

In [13]:
response = query_engine.query(eval_quick_test)
print(f"Question:{eval_quick_test}{chr(10)}")
print(f"Response:{chr(10)}{response.response}{chr(10)}")

text_md = ""
for n in response.source_nodes:
    
    text_md += (
        f"**Node ID:** {n.node.node_id}{chr(10)}"
        f"**Similarity:** {n.score}{chr(10)}"
        f"**Text:** {n.node.get_content()}{chr(10)}"
        f"**Metadata:** {n.node.metadata}{chr(10)}"
        f"~~~~{chr(10)}"
    )
print(text_md)

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
Question:My disabled daughter is 28 years old. Is she covered?

Response:
According to the information provided, a "Covered Family Member" includes any children of an Employee who are incapable of self-support due to physical and/or developmental disabilities and who are dependent on the Employee for support, regardless of their age. However, coverage for a disabled child over the age of 26 will cease if they are found to no longer be totally and permanently disabled. 

Therefore, as your daughter is over the age of 26, and assuming she is no longer totally and permanently disabled, she would not be covered.

**Node ID:** node-2
**Similarity:** 0.03279569892473118
**Text:** ITPEU Benefits > Health &

- Read the evalution question set (along with expected answers)
- This is structured in Llamaindex's format for batch evaluations
- Also, load into a data frame (which we will write back to an excel file with responses, evaluations etc.)

In [14]:
with open(eval_questions, 'r') as file:
    data = pd.read_json(file)
     
    queries_df = pd.DataFrame(list(data['queries'].items()), columns=['query_num', 'query'])
    responses_df = pd.DataFrame(list(data['responses'].items()), columns=['query_num', 'expected_answer'])
    
    responses_df = pd.merge(queries_df, responses_df, on='query_num')


Send questions to engine in bulk

In [15]:
from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import QueryResponseDataset

eval_dataset = QueryResponseDataset.from_json(eval_questions)
eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]
pred_responses = get_responses(
    eval_qs, query_engine, show_progress=True
)
pred_response_strs = [str(p) for p in pred_responses]

c:\Users\sthan\anaconda3\Lib\site-packages\llama_index\core\evaluation\dataset_generation.py:109: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return cls(**data)
  0%|          | 0/6 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai

 17%|█▋        | 1/6 [00:01<00:08,  1.72s/it]

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"


 33%|███▎      | 2/6 [00:01<00:03,  1.28it/s]

...

 50%|█████     | 3/6 [00:02<00:01,  1.84it/s]

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"


 83%|████████▎ | 5/6 [00:02<00:00,  3.73it/s]

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"


100%|██████████| 6/6 [00:02<00:00,  2.27it/s]


In [16]:
from evaluation_utils import get_eval_results_df, get_summary_scores_df, get_answers_source_nodes

answers, sources = get_answers_source_nodes(pred_responses)

responses_df['generated_answer'] = answers

sources_df = pd.DataFrame()
sources_df['query_num'] = responses_df['query_num']
sources_df['query'] = responses_df['query']
sources_df = sources_df.join(pd.DataFrame(sources)[0].str.split("~~~~", expand=True))

tokencount_df['answer_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [17]:
with pd.ExcelWriter(output_file) as writer:
   responses_df.to_excel(writer, sheet_name="Responses", index=False)
   sources_df.to_excel(writer, sheet_name="Sources", index=False)
   tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)

Choose the LLM for evaluations

In [18]:
evaluation_llm_family = os.environ["EVALUATION_LLM_FAMILY"]
evaluation_llm_model = os.environ["EVALUATION_LLM_MODEL"]

if evaluation_llm_family == "OPENAI":
    Settings.eval_llm = OpenAI(temperature=0, model=evaluation_llm_model)
elif evaluation_llm_family == "COHERE":
    Settings.eval_llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=evaluation_llm_model)

Set up Evaluations

In [19]:
from llama_index.core.evaluation import QueryResponseDataset
from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import BatchEvalRunner

from llama_index.core.evaluation import (
    CorrectnessEvaluator,
)
from deepeval.integrations.llama_index import (
    DeepEvalAnswerRelevancyEvaluator,
    DeepEvalFaithfulnessEvaluator,
    DeepEvalContextualRelevancyEvaluator,
    DeepEvalBiasEvaluator,
    DeepEvalToxicityEvaluator,
)

eval_lidx_c = CorrectnessEvaluator(llm=Settings.eval_llm)
eval_deval_f = DeepEvalFaithfulnessEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_ar = DeepEvalAnswerRelevancyEvaluator( threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_cr = DeepEvalContextualRelevancyEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_b = DeepEvalBiasEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_t = DeepEvalToxicityEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)

c:\Users\sthan\anaconda3\Lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.13, however version 0.21.24 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [20]:
# For large eval sets (30+ questions)
evaluator_dict_essential = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f
}

# For troubleshooting 
evaluator_dict_extended = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Context_Relevancy": eval_deval_cr
}

# For small sets (< 10 questions)
evaluator_dict_full = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Answer_Relevancy": eval_deval_ar,
    "Context_Relevancy": eval_deval_cr,
    "Bias": eval_deval_b,
    "Toxicity": eval_deval_t ,
}

# Pick the list of evaluators to run
evaluator_dict = evaluator_dict_essential

# Make sure this list matches the chosenevaluator_dict 
evaluators = ["Correctness", "Faithfulness" ] 

batch_runner = BatchEvalRunner(evaluator_dict, workers=8)

In [21]:
eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs,
    responses=pred_responses,
    reference=ref_response_strs,
)

Output()

Output()

Output()

Output()

Output()

Output()

In [22]:
from evaluation_utils import get_eval_results_df, get_summary_scores_df

In [23]:
mean_df, sum_df = get_summary_scores_df(
    [eval_results ],
    [rag_strategy],
    evaluators
)

In [24]:
if "Correctness" in evaluators:
    correctness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Correctness"]
    )
    responses_df['correctness'] = correctness_df['score']

if "Faithfulness" in evaluators:
    faithfulness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Faithfulness"]
    )
    responses_df['faithfulness'] = faithfulness_df['score']

if "Answer_Relevancy" in evaluators:
    answer_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Answer_Relevancy"]
    )
    responses_df['answer_relevancy'] = answer_relevancy_df['score']

if "Context_Relevancy" in evaluators:
    context_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Context_Relevancy"]
    )
    responses_df['context_relevancy'] = context_relevancy_df['score']

if "Bias" in evaluators:
    bias_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Bias"]
    )
    responses_df['bias'] = bias_df['score']

if "Toxicity" in evaluators:
    toxicity_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Toxicity"]
    )
    responses_df['toxicity'] = toxicity_df['score']



In [25]:
responses_df['rag_strategy'] = rag_strategy
responses_df['rag_strategy_desc'] = rag_strategy_desc
responses_df['parameter_1'] = chunk_size
responses_df['parameter_2'] = similarity_top_k
responses_df['parameter_3'] = retriever_weights_string
responses_df['parameter_4'] = ""
responses_df['parameter_5'] = ""
responses_df['model'] = generation_llm_model 
responses_df['embed_model'] = embedding_llm_model 
responses_df['eval_model'] = evaluation_llm_model
responses_df['reranker'] = fusion_reranker
responses_df['run_date'] = datetime.today().strftime('%Y-%m-%d') 
responses_df['eval_name'] = eval_name

In [26]:
tokencount_df['eval_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [27]:
with pd.ExcelWriter(output_file) as writer:
   responses_df.to_excel(writer, sheet_name="Responses", index=False)
   sources_df.to_excel(writer, sheet_name="Sources", index=False)
   
   sum_df.to_excel(writer, sheet_name="Summary", index=False, startrow=0 , startcol=0)
   mean_df.to_excel(writer, sheet_name="Summary", index=False,startrow=5, startcol=0)
   
  
   if "Correctness" in evaluators:
      correctness_df.to_excel(writer, sheet_name="Correctness", index=False)
   
   if "Faithfulness" in evaluators:
      faithfulness_df.to_excel(writer, sheet_name="Faithfulness", index=False)

   if "Context_Relevancy" in evaluators:
      context_relevancy_df.to_excel(writer, sheet_name="Context_Relevancy", index=False)
   
   if "Answer_Relevancy" in evaluators:
      answer_relevancy_df.to_excel(writer, sheet_name="Answer_Relevancy", index=False)
   
   if "Bias" in evaluators:
      bias_df.to_excel(writer, sheet_name="Bias", index=False)
   
   if "Toxicity" in evaluators:
      toxicity_df.to_excel(writer, sheet_name="Toxicity", index=False)
   
   tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)